In [21]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to Generative AI functions in BigQuery

## Overview

This tutorial will guide you through powerful generative AI capabilities available in BigQuery. You'll get hands-on experience using part of the suite of generative **`AI.*` functions** that integrate directly with powerful models like Gemini. This allows you to perform sophisticated AI-driven analysis on your data right within your familiar SQL environment.

While the examples in this notebook will focus on **text inputs** to the generative AI models, many of these capabilities extend to **multi-modal analysis**.

### Objectives

We'll cover how to:

* **Prompt models to generate text and structured data** with `AI.GENERATE` and `AI.GENERATE_TABLE`


---

## 1. Before you begin

### 1.1. Set your project ID

In [22]:
PROJECT_ID_LIST=!gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID=PROJECT_ID_LIST[0]
LOCATION="us-central1"
DATA_DS="rscw_oltp_stg_ds"
MULTIMODAL_STAGE_DS="rscw_multimodal_ai_stg_ds"

### 1.2. Create a BigQuery Dataset

Running the following query creates a BigQuery dataset called `rscw_multimodal_ai_stg_ds` to house tables created in this tutorial:

In [23]:
%%bigquery --project {PROJECT_ID}

CREATE SCHEMA `rscw_multimodal_ai_stg_ds` OPTIONS (location = 'us-central1');

Executing query with job ID: d304cbed-c8cd-4f18-9490-794b3570972b
Query executing: 0.23s


ERROR:
 409 Already Exists: Dataset qwiklabs-gcp-04-e2a62eed5a2c:rscw_multimodal_ai_stg_ds; reason: duplicate, message: Already Exists: Dataset qwiklabs-gcp-04-e2a62eed5a2c:rscw_multimodal_ai_stg_ds

Location: us-central1
Job ID: d304cbed-c8cd-4f18-9490-794b3570972b



### 1.3. Create BigQuery Cloud resource connection

You will need to create a [Cloud resource connection](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) to enable BigQuery to interact with Vertex AI services.

In [24]:
!bq mk --connection --location=us-central1 \
    --connection_type=CLOUD_RESOURCE rscw_cloud_resource_connection_usc

BigQuery error in mk operation: Already Exists: Connection
projects/605974140031/locations/us-
central1/connections/rscw_cloud_resource_connection_usc


### 1.4. Set permissions for Service Account

The resource connection service account requires certain project-level permissions to interact with Vertex AI.

In [25]:
SERVICE_ACCT = !bq show --format=prettyjson --connection us-central1.rscw_cloud_resource_connection_usc | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

bqcx-605974140031-ithh@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [26]:
import time

!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/aiplatform.user'

# wait 60 seconds, give IAM updates time to propagate, otherwise, following cells will fail
time.sleep(60)

Updated IAM policy for project [qwiklabs-gcp-04-e2a62eed5a2c].
ERROR: (gcloud.projects.add-iam-policy-binding) Resource in projects [qwiklabs-gcp-04-e2a62eed5a2c:setIamPolicy] is the subject of a conflict: There were concurrent policy changes. Please retry the whole read-modify-write with exponential backoff. The request's ETag '\007\006J\223]`\366\'' did not match the current policy's ETag '\007\006J\223]pO\332'.


---

## 2. Generate text and structured data with `AI.GENERATE` and `AI.GENERATE_TABLE`

These functions allow you to leverage the power of large language models (LLMs) directly within BigQuery to generate new text content, including creating content with a specified schema. Both functions work by sending requests to your choice of a [generally available](https://cloud.google.com/vertex-ai/generative-ai/docs/models#generally_available_models) or [preview](https://cloud.google.com/vertex-ai/generative-ai/docs/models#preview_models) Gemini model, and then returning that model's response.

### 2.1. Using `AI.GENERATE`: Generate product descriptions

Let's use the [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate) function to generate product descriptions for the products in the `rscw_oltp_stg_ds.product_master` table and persist to `rscw_multimodal_ai_stg_ds.product_master_augmented` table. We will also generate some fake reviews of the products for subsequent sections.

In [27]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_master_augmented` AS
SELECT
  product_id,
  product_nm,
  AI.GENERATE(('Generate a product description for this product: ', product_nm),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    endpoint => 'gemini-2.5-flash').result AS product_description,
  AI.GENERATE(('Generate a 70-30 balance of positive and negative reviews for the product, generate 10 reviews and as json called reviews but without any backticks and without the word json in the review payload. Also, the review should have just a rating and a comment: ', product_nm),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    endpoint => 'gemini-2.5-flash').result AS product_reviews,
  '' AS product_image_gcs_uri,
  '' AS product_user_manual_gcs_uri
FROM
  `rscw_oltp_stg_ds.product_master`
WHERE
  category_nm = 'home_appliances'
AND
  product_id in ('c2f2d10f8b047f5cffef731479a34432' ,'e9aa3fd3f316b44c85f671552a04fa4a' ,'d418ddef3e7017381084956220b00b9d' ,'e403a54af54730eb4f3e8a346d943579' ,'ba9a847a2c023953743ba9e5ef6c88c6' ,'832137af5164ffb89cf407d686aa2d0d' ,'900d210e9ea6ee40dd0290df76bde9fc' ,'520ac02e0e8cdf179eed22f204e3f26d' ,'aa15d9042b3749f119a734d7df2b68bc' ,'ccb125d39320e9d900107ce4109589dd')
LIMIT 10;


SELECT product_id, product_nm,product_description
FROM rscw_multimodal_ai_stg_ds.product_master_augmented LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_nm,product_description
0,ba9a847a2c023953743ba9e5ef6c88c6,Portable Fabric Steamer,"## Effortless Style, On-the-Go: Your Portable ..."
1,520ac02e0e8cdf179eed22f204e3f26d,Anti-Fatigue Kitchen Mat,## Reclaim Your Comfort: The Premium Anti-Fati...


### 2.2. Using `AI.GENERATE`: Extract keywords from reviews

Let's use the [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate) function to extract keywords from the product reviews in the `rscw_multimodal_ai_stg_ds.product_master_augmented` table.

To do this, lets first flatten the product_reviews json for each product into a separate table `rscw_multimodal_ai_stg_ds.product_reviews`

In [28]:
%%bigquery --project {PROJECT_ID}

-- 1) Unnest the ratings JSON into rows

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_reviews` AS
SELECT t.product_id, t.product_nm, JSON_EXTRACT_SCALAR(review, '$.rating') AS review_rating,JSON_EXTRACT_SCALAR(review, '$.comment') AS review_comment
FROM
    rscw_multimodal_ai_stg_ds.product_master_augmented  AS t,
    UNNEST(JSON_EXTRACT_ARRAY(t.product_reviews, '$.reviews')) AS review;

SELECT product_id, product_nm,review_rating, review_comment
FROM rscw_multimodal_ai_stg_ds.product_reviews LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_nm,review_rating,review_comment
0,d418ddef3e7017381084956220b00b9d,SealFresh Bag Sealer,5,This SealFresh Bag Sealer is fantastic! Great ...
1,e9aa3fd3f316b44c85f671552a04fa4a,QuickPress Mini Iron,4,Handy for crafting and sewing projects. It's l...


In [29]:
%%bigquery --project {PROJECT_ID}

--2a) Extract keywords

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    endpoint => 'gemini-2.5-flash').result AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,I never thought a mat could make such a differ...,"mat, difference, stays in place, reduced strai..."
1,520ac02e0e8cdf179eed22f204e3f26d,I was really hoping this mat would help with m...,"mat, foot pain, no difference, uncomfortable, ..."


You can see that BigQuery has prompted the specified model for each row of the data and returned the response.

#### Using arguments to adjust model configuration
In addition to the `prompt`, `connection_id`, and `endpoint`, there are additional arguments available in `AI.GENERATE` at your disposal for customizing your generation request.

##### The `model_params` argument
There are several parameters you can specify within the `model_params` argument. You can read more about which specific parameters are available for `AI.GENERATE` in the [documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate#arguments). The next query adds custom `temperature` and `maxOutputTokens` parameters.

* The `temperature` parameter controls the randomness of the response, where a lower value makes the output more predictable and focused, while a higher value encourages more creative and diverse results.

* The `maxOutputTokens` parameter sets the maximum length of the model's response by limiting the total number of tokens (pieces of words) it can generate.

You can read more about the various parameters [here](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#generationconfig), and the parameters' default values are documented in the model card in the [model documentation for Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/models).

In [30]:
%%bigquery --project {PROJECT_ID}

--2b) Extract keywords and specify temperature and max output tokens

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000}}',
    endpoint => 'gemini-2.5-flash').result AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,I never thought a mat could make such a differ...,"mat, difference, stays in place, reduced strai..."
1,520ac02e0e8cdf179eed22f204e3f26d,"Very comfortable underfoot, offering good supp...",comfortable underfoot\ngood support\nlengthy c...


Why are some of the responses empty? By setting the `maxOutputTokens` value to 1,000 (much lower than the default 65,535 which was used in the first queries), you are seeing this limit in action. Gemini 2.5 Flash has [thinking capabilities](https://cloud.google.com/vertex-ai/generative-ai/docs/thinking), which means the model goes through a "thinking process" before determining and returning its response, and this thinking process consumes output tokens. In this case, the model is hitting its `maxOutputTokens` value before it moves from its thinking process to returning the actual response.

You can resolve this issue by increasing the `maxOutputTokens` value and/or setting a `thinking_budget` to limit the amount of tokens that can be consumed by the thinking process.

##### Setting a `thinking_budget`

You can set a [`thinking_budget`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate#thinking-budget) to manage how many output tokens are allocated to the thinking phase of the model's response development.

The next query shows how to set the `thinking_budget` as an additional parameter in the `model_params` argument.

In [31]:
%%bigquery --project {PROJECT_ID}

--2c) Extract keywords, specify temperature, max output tokens, and a specific thinking budget

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000, "thinking_config": {"thinking_budget": 500}}}',
    endpoint => 'gemini-2.5-flash').result AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;


Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,"The comfort level is superb, and I can definit...","comfort level, superb, less strain, legs, long..."
1,520ac02e0e8cdf179eed22f204e3f26d,Provides excellent cushioning and support. Whi...,"cushioning, support, fatigue, colors, decor"


##### The `output_schema` argument

As you can see in the last query, the model's responses were not provided in a standard structure. You can use the **`output_schema`** argument to define the format of the responses.

First, you'll specify the output as a string of key words by adding `output_schema => 'keywords ARRAY<STRING>'` to the `AI.GENERATE` request.

In [32]:
%%bigquery --project {PROJECT_ID}


--2d) Extract keywords, specify temperature, max output tokens, a specific thinking budget & with output schema

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000, "thinking_config": {"thinking_budget": 500}}}',
    endpoint => 'gemini-2.5-flash',
    output_schema => 'keywords ARRAY<STRING>'
    ).keywords AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;


Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,"Absolutely love this mat! It's thick, supporti...","[mat, thick, supportive, non-slip, feet comfor..."
1,520ac02e0e8cdf179eed22f204e3f26d,This mat is a tripping hazard! It slides all o...,"[mat, tripping hazard, slides, tiled kitchen f..."


Now the responses are now arrays of keywords - a more useful format for this use case!

Next, you'll adjust the prompt to the model to extract the sentiment as well, and adjust the `output_schema` accordingly.

In [33]:
%%bigquery --project {PROJECT_ID}

--2d) Extract keywords, specify temperature, max output tokens, and a specific thinking budget
--    and with sentiment as well

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords_sentiment` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the review_comment; Return both keywords and sentiment', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000, "thinking_config": {"thinking_budget": 500}}}',
    endpoint => 'gemini-2.5-flash',
    output_schema => 'keywords ARRAY<STRING>, sentiment STRING').full_response
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,full_response
FROM rscw_multimodal_ai_stg_ds.product_review_keywords_sentiment LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,full_response
0,e9aa3fd3f316b44c85f671552a04fa4a,Great for small ironing tasks and delicate fab...,"{""candidates"":[{""avg_logprobs"":-0.432375272115..."
1,900d210e9ea6ee40dd0290df76bde9fc,Broke after only a few uses. Very disappointed...,"{""candidates"":[{""avg_logprobs"":-0.344933509826..."


This time, the response is in a JSON format. Why?

When using `output_schema` with [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate), you can specify in the SQL which field you want to return. In this case the SQL specifies the `.full_response` which will provide a JSON object containing the various text response elements. You may alternatively specify to return any one of the schema columns you defined, such as `.keywords` (as was used in the first query of this section) or `.sentiment`. However, you cannot specify to return multiple columns.

 You could use SQL to parse the JSON provided in the `.full_response`,  but for ease of use, let's look at the [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) function.

### 2.3. Using `AI.GENERATE_TABLE`: Extract keywords and sentiment from product reviews

The [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) function provides expanded functionality for turning unstructured text into a structured table. Here, you'll perform the same analysis to extract keywords and overall sentiment from the `rscw_multimodal_ai_stg_ds.product_review` table and return the response as multiple columns in one query.

#### Note:

The syntax for using [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) differs from [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate), and it requires that you first create a remote model in BigQuery that connects to the one of the [generally available](https://cloud.google.com/vertex-ai/generative-ai/docs/models#generally_available_models) or [preview](https://cloud.google.com/vertex-ai/generative-ai/docs/models#preview_models) Gemini models.

We will house the remote model in the BigQuery dataset  `rscw_multimodal_ai_stg_ds`

#### 2.3.1. Create a remote model for Gemini 2.5 Flash

Now you'll create the remote model, which is simply a pointer to the model endpoint. In this example, you'll continue to use the `gemini-2.5-flash` model endpoint.

Once you create this remote model, it is saved in your `rscw_multimodal_ai_stg_ds` dataset so that you can use it in any future analysis.

In [34]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE MODEL
  `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`
REMOTE WITH
    CONNECTION `us-central1.rscw_cloud_resource_connection_usc`
    OPTIONS (ENDPOINT = 'gemini-2.5-flash');


Query is running:   0%|          |

""


#### 2.3.2. Run analysis using the remote model

Now that the remote model has been created, you can prompt it with [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) to analyze data and specify the output schema with multiple columns returned.

In [35]:
%%bigquery --project {PROJECT_ID}


SELECT
  product_id,
  review_comment,
  keywords,
  sentiment
FROM
  AI.GENERATE_TABLE(
    MODEL `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`,
    (
      SELECT
        product_id,
        review_comment,
        ('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the following review:',review_comment) AS prompt,
      FROM
        `rscw_multimodal_ai_stg_ds.product_reviews`
    ),
    STRUCT(
      'keywords ARRAY<STRING>, sentiment STRING' AS output_schema
    )
  );

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords,sentiment
0,e9aa3fd3f316b44c85f671552a04fa4a,Does a decent job for most everyday clothing. ...,"[decent job, everyday clothing, blouses, shirts]",Positive
1,832137af5164ffb89cf407d686aa2d0d,Very disappointed. This unit stopped sealing p...,"[disappointed, unit, sealing, vacuum function,...",Negative
2,e403a54af54730eb4f3e8a346d943579,Good value for the price. It blends well for m...,"[Good value, blends well, noisy, highly recomm...",Positive
3,ccb125d39320e9d900107ce4109589dd,Absolutely perfect for quick clean-ups in the ...,"[quick clean-ups, car, house, Lightweight, eff...",Positive
4,ccb125d39320e9d900107ce4109589dd,The suction is much weaker than advertised. It...,"[suction, weaker, struggles, dust, bin, fills ...",Negative
...,...,...,...,...
75,aa15d9042b3749f119a734d7df2b68bc,Love the varied speed options. It starts slow ...,"[varied speed options, prevent splashes, power...",Positive
76,520ac02e0e8cdf179eed22f204e3f26d,This mat is a tripping hazard! It slides all o...,"[tripping hazard, slides, tiled kitchen floor,...",Negative
77,520ac02e0e8cdf179eed22f204e3f26d,I never thought a mat could make such a differ...,"[mat, stays in place, reduced strain on joints...",Positive
78,832137af5164ffb89cf407d686aa2d0d,Solid performance from the Handy-Helper. It oc...,"[Solid performance, Handy-Helper, second attem...",Positive


#### 2.3.3. Saving results in a BigQuery table
Wrapping any `SELECT` statement with the [`CREATE TABLE` statement](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-definition-language#create_table_statement) allows you to create a permanent table from your query results. You can run the next two queries to see that in action.

In [36]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.products_reviews_keywords_sentiment` AS (
  SELECT
    product_id,
    review_comment,
    keywords,
    sentiment
  FROM
    AI.GENERATE_TABLE(
      MODEL `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`,
      (
        SELECT
          product_id,
          review_comment,
          ('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the following review:',review_comment) AS prompt,
        FROM
          `rscw_multimodal_ai_stg_ds.product_reviews`
      ),
      STRUCT(
        'keywords ARRAY<STRING>, sentiment STRING' AS output_schema)));

Query is running:   0%|          |

""


In [37]:
%%bigquery --project {PROJECT_ID}

SELECT *
FROM `rscw_multimodal_ai_stg_ds.products_reviews_keywords_sentiment` LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords,sentiment
0,520ac02e0e8cdf179eed22f204e3f26d,This mat is a tripping hazard! It slides all o...,"[tripping hazard, slides, tiled kitchen floor,...",Negative
1,520ac02e0e8cdf179eed22f204e3f26d,I was really hoping this mat would help with m...,"[mat, foot pain, difference, comfortable, bare...",Negative
2,520ac02e0e8cdf179eed22f204e3f26d,The material feels rather cheap and it started...,"[material feels cheap, flatten out significant...",Negative
3,832137af5164ffb89cf407d686aa2d0d,The suction power on this sealer isn't always ...,"[suction power, sealer, bags, leak air, incons...",Negative
4,832137af5164ffb89cf407d686aa2d0d,This food sealer is far too bulky for my small...,"[food sealer, bulky, small kitchen, expensive ...",Negative
5,832137af5164ffb89cf407d686aa2d0d,Very disappointed. This unit stopped sealing p...,"[disappointed, unit stopped sealing, light use...",Negative
6,900d210e9ea6ee40dd0290df76bde9fc,Found it a bit flimsy for thicker batters. The...,"[flimsy, thicker batters, handle, cheap]",Negative
7,900d210e9ea6ee40dd0290df76bde9fc,Broke after only a few uses. Very disappointed...,"[Broke, disappointed, durability]",Negative
8,aa15d9042b3749f119a734d7df2b68bc,Disappointed with the build quality. The attac...,"[build quality, attachments, flimsy, bend, not...",Negative
9,aa15d9042b3749f119a734d7df2b68bc,"While it's okay for light tasks, this mixer re...","[light tasks, mixer, struggles, cake batter, d...",Negative


#### 2.3.4. Using arguments to adjust model configuration
Similar to `AI.GENERATE`, [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) also allows you to customize the parameters for your generation request. The placement of these parameters is within the `STRUCT` value. The [documentation page](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table#:~:text=dataset.table%60%20%7C%20(query_statement)%20%7D%2C-,STRUCT(,-output_schema%20AS%20output_schema) contains the available fields you can define within the `STRUCT`.

Next you'll take the same query and now define values for `temperature` and `max_output_tokens`.

In [ ]:
%%bigquery --project {PROJECT_ID}

SELECT
    product_id,
    review_comment,
    keywords,
    sentiment
  FROM
    AI.GENERATE_TABLE(
      MODEL `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`,
      (
        SELECT
          product_id,
          review_comment,
          ('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the following review:',review_comment) AS prompt,
        FROM
          `rscw_multimodal_ai_stg_ds.product_reviews`
      ),
      STRUCT(
        'keywords ARRAY<STRING>, sentiment STRING' AS Output_schema,
      0.5 AS temperature,
      1000 AS max_output_tokens));

Executing query with job ID: f8344582-8a49-4609-a03a-23f163f4c0d0
Query executing: 51.17s



### This concludes the task. Please proceed to the lab instructions page for the next task.